DynaFlow is a new steady-state simulation tool that aims at calculating the steady-state point by using a simplified time-domain simulation, guaranteeing the correctness of the solution found. 
This tutorial aims at illustrating the interest of Dynaflow compared to a fully static simulator.

The system is made of a generator, two phase shifter transformers in parallel, a transformer, five lines and a load.

We simulate the opening of a line at t = 30s with different scenarios on the time constants of the phase shifters:
1) The two phase shifters have the following time constants: t1st = 40s and tNext = 20s.
2) One phase shifter has the following time constants: t1st = 20s and tNext= 10s, the other one: t1st = 40s and tNext = 20s.

1/ DYNAFLOW installing

In [ ]:
!curl -L $(curl -s -L -X GET https://api.github.com/repos/dynawo/dynaflow-launcher/releases/latest | grep "DynaFlowLauncher_Linux" | grep url | cut -d '"' -f 4) -o DynaflowLauncher_Linux_latest.zip
!unzip DynaflowLauncher_Linux_latest.zip > /dev/null 2>&1
!./dynaflow-launcher/dynaflow-launcher.sh help

2/ POWSYBL installing and setting of default parameter (path to dynaflow_launcher)

In [ ]:
pip install pypowsybl > /dev/null 2>&1

In [ ]:
!mkdir /home/jovyan/.itools/
!cp /home/jovyan/Dynaflow_notebook/config.yml /home/jovyan/.itools/
!cat /home/jovyan/.itools/config.yml

3/ NOTEBOOK

In [ ]:
import pypowsybl
import pypowsybl.network as pn
import pypowsybl.loadflow as lf
import pypowsybl.security as sa
import os.path

In [2]:
# Load initial situation
network_file = 'PhaseShifters.iidm'
network = pn.load(network_file)
network.get_network_area_diagram()

In [3]:
# Initial tap positions
network.get_phase_tap_changers()

,tap,low_tap,high_tap,step_count,regulating,regulation_mode,regulation_value,target_deadband,regulating_bus_id
id,,,,,,,,,
_BUS____5-BUS____6-1_PS,13,1,25,25,True,CURRENT_LIMITER,770.0,0.0,_BUS____5_VL_0
_BUS____5-BUS____7-2_PS,13,1,25,25,True,CURRENT_LIMITER,770.0,0.0,_BUS____5_VL_0


In [6]:
# 1st Dynaflow simulation: identical time constants (defined in file "setting-1.xml")
p = lf.Parameters(provider_parameters={'svcRegulationOn' : 'true',
                                       'shuntRegulationOn' : 'true',
                                       'automaticSlackBusOn' : 'true',
                                       'dsoVoltageLevel' : '63.0',
                                       'activePowerCompensation' : 'P',
                                       'settingPath' : os.getcwd() + '/setting-1.xml',
                                       'assemblingPath' : os.getcwd() + '/assembling.xml',
                                       'stopTime' : '300',
                                       'chosenOutputs' : 'STEADYSTATE, LOSTEQ, TIMELINE, CONSTRAINTS',
                                       'timeStep' : '2.6'})
lf.run_ac(network, parameters=p, provider='DynaFlow')
network1 = network

PyPowsyblError: com.powsybl.commons.PowsyblException: Error during the execution in directory  /home/pachecam/tmp/dynaflow_12102227360503391858 exit codes: Task 0 : 134

In [14]:
# Final tap positions after N-1 simulation with identical time constants
network1.get_phase_tap_changers()

,tap,low_tap,high_tap,step_count,regulating,regulation_mode,regulation_value,target_deadband,regulating_bus_id
id,,,,,,,,,
_BUS____5-BUS____6-1_PS,18,1,25,25,True,CURRENT_LIMITER,770.0,0.0,_BUS____5_VL_0
_BUS____5-BUS____7-2_PS,18,1,25,25,True,CURRENT_LIMITER,770.0,0.0,_BUS____5_VL_0


In [15]:
# 2nd Dynaflow simulation: different time constants (defined in file "setting-2.xml")
network = pn.load(network_file)
p = lf.Parameters(provider_parameters={'svcRegulationOn' : 'true',
                                       'shuntRegulationOn' : 'true',
                                       'automaticSlackBusOn' : 'true',
                                       'dsoVoltageLevel' : '63.0',
                                       'activePowerCompensation' : 'P',
                                       'settingPath' : os.getcwd() + '/setting-2.xml',
                                       'assemblingPath' : os.getcwd() + '/assembling.xml',
                                       'stopTime' : '300',
                                       'chosenOutputs' : 'STEADYSTATE, LOSTEQ, TIMELINE, CONSTRAINTS',
                                       'timeStep' : '2.6'})
lf.run_ac(network, parameters=p, provider='DynaFlow')
network2 = network

In [16]:
# Final tap positions after N-1 simulation with different time constants
network2.get_phase_tap_changers()

,tap,low_tap,high_tap,step_count,regulating,regulation_mode,regulation_value,target_deadband,regulating_bus_id
id,,,,,,,,,
_BUS____5-BUS____6-1_PS,18,1,25,25,True,CURRENT_LIMITER,770.0,0.0,_BUS____5_VL_0
_BUS____5-BUS____7-2_PS,17,1,25,25,True,CURRENT_LIMITER,770.0,0.0,_BUS____5_VL_0


In [17]:
print('Transits in transformers with identical time constants:')
print(network1.get_2_windings_transformers()[['i1','i2']])
print('\nTransits in transformers with different time constants:')
print(network2.get_2_windings_transformers()[['i1','i2']])

Transits in transformers with identical time constants:
                                 i1         i2
id                                            
_BUS____5-BUS____6-1_PS  233.952319  39.468929
_BUS____5-BUS____7-2_PS  233.952319  39.468929
_BUS____5-BUS____8-1_PT  383.855389  64.758329

Transits in transformers with different time constants:
                                 i1         i2
id                                            
_BUS____5-BUS____6-1_PS  223.959508  37.783092
_BUS____5-BUS____7-2_PS  253.934704  42.840058
_BUS____5-BUS____8-1_PT  373.860516  63.072144


The currents in both phase shifters are compared in both cases. Depending on the time constants of the phase shifter, the steady-state after the event is different.
This cannot be captured by a fully static simulator, in which all the phase shifters are activated at the same time with an external loop.